![ss](images/Spark核心RDD.jpg)

    1.PARTITION是RDD的基本单位,不同的处理阶段产生了依赖关系的PARTITION,依赖关系体现了SPARK的弹性,并且参与数据的容错恢复.
    2.各个PARTITION的处理是并行的,一个PARTITION只能对于一个TASK,由一个核心处理
    3.从一种PARTITION,变成另一种PARTITION的操作称为一个算子!
    4.Partition可以分布在多个节点上

## standalone模式下面的Spark执行原理
![ss](images/spark_standalone.png)

    1.Driver是standalone模式下的 任务 主节点,一个JVM进程
    2.Worker是standalone模式下的 从节点,一个JVM进程
    3.Driver发放任务到worker,汇集worker的计算结果,谨防OOM
    4,master是集群的管理者,汇集集群中的资源(worker信息)
   

## 基于standalone资源管理 集群搭建
    standalone是spark自带的资源调度框架
    
    1.conf/slaves:加入node2,node3,node4
    2.conf/spark-env.sh:修改如下变量
        export SPARK_MASTER_IP=node1
        export SPARK_MASTER_PORT=7077
        export SPARK_WORKER_CORES=2
        export SPARK_WORKER_MEMORY=2g
        export SPARK_MASTER_WEBUI_PORT=8888
        //主MasterIP,port
        //每个worker使用2核心,2G内存
        //web接口是8888
    3.在主master 启动全部节点:
        sbin/start-all.sh
        sbin/stop-all.sh
    4.提交作业
        有2种方式:
        //客户端方式,你的客户端就是driver!
        bin/spark-submit --master spark://node1:7077 --class main.spark.app.SparkPI myzxk.jar
        //集群方式,集群中一个worker被认为是driver
        bin/spark-submit --deploy-mode cluster --master spark://node1:7077 --class main.spark.app.SparkPI myzxk.jar
        
        (org.apache.spark.examples.SparkPi)

![](images/Standalone-client.jpg)

## 备注: Driver和worker进行大量的信息交互,汇集worker的计算结果,比如,RDD.collect(),把Work的数据拉到Driver,所有会造成client网络拥堵

![](images/Standalone-cluster.jpg)
## 集群模式,Driver是在集群中的随机一台机器上启动,所以客户端不会有网络拥堵的压力

## 基于yarn资源调度管理

    A.集群配置
        conf/spark-env.sh  或者/conf/spark-default.conf:修改如下变量
        HADOOP_CONF_DIR=HADOOP_HOME/etc/hadoop
    B.yarn资源调度处理任务的流程
        回顾下map-reduce,yarn调度:
        1.yarn构架 分为resourceManager,NodeManager,resourceManager协调,管理,监控NodeManager,NodeManager节点
        上面跑实际的 任务
        2.job被提交给resourceManager后,resourceManager随机挑选一台NodeManager,创建APP-MASTER(可以认为是java对象),
        app_master 创建,跟踪,作业
        3.app_master要求resourceManager,在NodeManager上分配资源,启动container(JVM),一个大job就这样在各个NodeManager
        节点的container(JVM)运行
    C.yarn-client的执行
        与map-reduce的区别:客户端是driver,app_master要求申请的不是container,而是executor,executor.threadpool与driver
        频繁的通信,结果collect 给客户端driver
        client-->ResourceManager-->applicationMaster-->申请创建executor-->executor.threadpool
            |                                                                        | 
            |                                                                        |
            driver<------------------------------------------------------------------>|
        命令:bin/spark-submit --master yarn --deploy-mode client --class main.spark.app.SparkPI myzxk.jar
    D.yarn-cluster的执行:
        与yarn-client的区别,client不再是driver,applicationMaster=driver
        命令:bin/spark-submit --master yarn --deploy-mode cluster --class main.spark.app.SparkPI myzxk.jar
  ![ss](images/Yarn-client模式提交任务.jpg)
  ![ss](images/Yarn-cluster模式提交任务.jpg)